# Estimating Gender Discrimination in the Workplace

Luopeiwen Yi

In this exercise we'll use data from the 2018 US Current Population Survey (CPS) to try and estimate the effect of being a woman on workplace compensation. 

Note that our focus will be *only* on differential compensation in the work place, and as a result it is important to bear in mind that our estimates are not estimates of *all* forms of gender discrimination. For example, these analyses will not account for things like gender discrimination in terms of *getting* jobs. We'll discuss this in more detail below.

In [92]:
import pandas as pd
import numpy as np
import warnings
import matplotlib.pyplot as plt
from statsmodels.stats.power import TTestIndPower
import statsmodels.api as sm
from scipy import stats
import statsmodels.formula.api as smf

warnings.filterwarnings("ignore")

pd.set_option("mode.copy_on_write", True)

## Exercise 1: 

Begin by downloading and importing 2018 CPS data from [https://github.com/nickeubank/MIDS_Data/tree/master/Current_Population_Survey](https://github.com/nickeubank/MIDS_Data/tree/master/Current_Population_Survey). The file is called `morg18.dta` and is a Stata dataset. Additional data on the dataset can be found by following the links in the README.txt file in the folder, but for the moment it is sufficient to know this is a national survey run in the United States.

The survey does include some survey weights we won't be using (i.e. not everyone in the sample was included with the same probability), so the numbers we estimate will not be perfect estimates of the gender wage gap in the United States, but they are pretty close.

In [93]:
# Read the data
morg18 = pd.read_stata("morg18.dta")

# Display the first few rows of the dataframe
morg18.head()

,hhid,intmonth,hurespli,hrhtype,minsamp,hrlonglk,hrsample,hrhhid2,serial,hhnum,...,ym_file,ym,ch02,ch35,ch613,ch1417,ch05,ihigrdc,docc00,dind02
0,000004795110719,January,1.0,Husband/wife primary fam (neither in Armed For...,MIS 8,MIS 2-4 Or MIS 6-8 (link To,0601,06011,1,1,...,696,681,0,0,0,0,0,14.0,NaN,NaN
1,000004795110719,January,1.0,Husband/wife primary fam (neither in Armed For...,MIS 8,MIS 2-4 Or MIS 6-8 (link To,0601,06011,1,1,...,696,681,0,0,0,0,0,13.0,NaN,NaN
2,000110339935453,January,1.0,Unmarried civilian female primary fam householder,MIS 4,MIS 2-4 Or MIS 6-8 (link To,0701,07011,1,1,...,696,693,0,0,0,1,0,12.0,Office and administrative support occupations,"Health care services , except hospitals"
3,000110339935453,January,1.0,Unmarried civilian female primary fam householder,MIS 4,MIS 2-4 Or MIS 6-8 (link To,0701,07011,1,1,...,696,693,0,0,0,0,0,12.0,Office and administrative support occupations,Administrative and support services
4,000110359424339,January,1.0,Unmarried civilian female primary fam householder,MIS 4,MIS 2-4 Or MIS 6-8 (link To,0711,07111,1,1,...,696,693,0,0,0,0,0,NaN,Healthcare practitioner and technical occupations,Hospitals


## Exercise 2:

Because our interest is only in-the-workplace wage discrimination among full-time workers, we need to start by subsetting our data for people currently employed (and "at work", not "absent") at the time of this survey using the `lfsr94` variable, who are employed full time (meaning that their usual hours per week—`uhourse`—is 35 or above).

As noted above, this analysis will miss many forms of gender discrimination. For example, in dropping anyone who isn't working, we immediately lose any women who couldn't get jobs, or who chose to lose the workforce because the wages they were offered (which were likely lower than those offered men) were lower than they were willing / could accept. And in focusing on full time employees, we miss the fact women may not be offered full time jobs at the same rate as men. 

In [94]:
# get unique values in lfsr94
unique_values_lfsr94 = morg18["lfsr94"].unique()

# Printing the unique values
print("The unique categories in the lfsr94 column are:", unique_values_lfsr94)

The unique categories in the lfsr94 column are: ['Disabled-Not In Labor Force', 'Retired-Not In Labor Force', 'Employed-At Work', 'Unemployed-Looking', 'Employed-Absent', 'Other-Not In Labor Force', NaN, 'Unemployed-On Layoff']
Categories (7, object): ['Employed-At Work' < 'Employed-Absent' < 'Unemployed-On Layoff' < 'Unemployed-Looking' < 'Retired-Not In Labor Force' < 'Disabled-Not In Labor Force' < 'Other-Not In Labor Force']


In [95]:
# get unique values in uhourse
unique_values_uhourse = morg18["uhourse"].unique()

# Printing the unique values
print("The unique categories in the uhourse column are:", unique_values_uhourse)

print("The data type of uhourse is:", morg18["uhourse"].dtype)

The unique categories in the uhourse column are: [nan 40. 30. -4. 36. 20. 48. 35. 44. 15. 50. 45. -1. 60. 17. 19. 41. 10.
 70. 25. 11. 75. 18. 32.  2. 80.  5. 24.  4.  3. 38.  6. 56. 96. 82. 55.
 52. 77. 84.  0. 12. 16. 42. 43. 54. 23. 53. 33. 28. 65.  9. 37. 72. 26.
  7. 61. 27. 99. 58.  8. 21. 22. 13. 47. 63.  1. 46. 39. 29. 14. 49. 66.
 34. 57. 90. 62. 85. 31. 87. 68. 69. 76. 83. 98. 88. 51. 78. 64. 67. 92.
 95. 74. 86. 81. 59. 71. 73. 91. 89. 94.]
The data type of uhourse is: float64


In [96]:
# Filter for individuals who are 'Employed-At Work' and who are full-time workers (35 hours or more per week)
employed_full_at_work = morg18[
    (morg18["lfsr94"] == "Employed-At Work") & (morg18["uhourse"] >= 35)
]

# Display the first few rows of the filtered DataFrame
employed_full_at_work.head()

,hhid,intmonth,hurespli,hrhtype,minsamp,hrlonglk,hrsample,hrhhid2,serial,hhnum,...,ym_file,ym,ch02,ch35,ch613,ch1417,ch05,ihigrdc,docc00,dind02
2,000110339935453,January,1.0,Unmarried civilian female primary fam householder,MIS 4,MIS 2-4 Or MIS 6-8 (link To,0701,07011,1,1,...,696,693,0,0,0,1,0,12.0,Office and administrative support occupations,"Health care services , except hospitals"
3,000110339935453,January,1.0,Unmarried civilian female primary fam householder,MIS 4,MIS 2-4 Or MIS 6-8 (link To,0701,07011,1,1,...,696,693,0,0,0,0,0,12.0,Office and administrative support occupations,Administrative and support services
4,000110359424339,January,1.0,Unmarried civilian female primary fam householder,MIS 4,MIS 2-4 Or MIS 6-8 (link To,0711,07111,1,1,...,696,693,0,0,0,0,0,NaN,Healthcare practitioner and technical occupations,Hospitals
6,000110651278174,January,1.0,Civilian male primary individual,MIS 8,MIS 2-4 Or MIS 6-8 (link To,0601,06011,1,1,...,696,681,0,0,0,0,0,12.0,Transportation and material moving occupations,Transportation and warehousing
17,007680515071194,January,1.0,Civilian male primary individual,MIS 8,MIS 2-4 Or MIS 6-8 (link To,0611,06112,2,2,...,696,681,0,0,0,0,0,12.0,Transportation and material moving occupations,Retail trade


## Exercise 3

Now let's estimate the basic wage gap for the United States!

Earnings per week worked can be found in the `earnwke` variable. Using the variable `sex` (1=Male, 2=Female), estimate the gender wage gap in terms of wages per hour worked!

(You may also find it helpful, for context, to estimate the average hourly pay by dividing weekly pay by `uhourse`.)

In [97]:
# Estimate the average hourly pay
employed_full_at_work["average_hourly_pay"] = (
    employed_full_at_work["earnwke"] / employed_full_at_work["uhourse"]
)

# Estimate the average hourly wage by sex
average_hourly_wage_by_sex = employed_full_at_work.groupby("sex")[
    "average_hourly_pay"
].mean()

# Estimate the basic wage gap
wage_gap = average_hourly_wage_by_sex[1] - average_hourly_wage_by_sex[2]

# Display the wage gap
print(
    f"The estimate basic wage gap for the United States",
    f"between male and female is around: ${round(wage_gap, 2)} per hour",
)

The estimate basic wage gap for the United States between male and female is around: $4.08 per hour


## Exercise 4

Assuming 48 work weeks in a year, calculate annual earnings for men and women. Report the difference in dollars and in percentage terms.

In [98]:
# Calculate annual pay
employed_full_at_work["annual_pay"] = employed_full_at_work["earnwke"] * 48
# Calculate annual pay for men
average_annual_pay_man = employed_full_at_work[employed_full_at_work["sex"] == 1][
    "annual_pay"
].mean()
# Calculate annual pay for women
average_annual_pay_woman = employed_full_at_work[employed_full_at_work["sex"] == 2][
    "annual_pay"
].mean()
# Calculate difference in dollars
annual_wage_gap = average_annual_pay_man - average_annual_pay_woman
# Calculate difference in percentage terms
annual_wage_gap_percentage = ((annual_wage_gap) / average_annual_pay_woman) * 100

print(
    f"The annual wage differences between male and female is around ${round(annual_wage_gap,2)}."
)
print(
    f"On average, men earn around {round(annual_wage_gap_percentage, 2)}% more than women annually."
)

The annual wage differences between male and female is around $10514.44.
On average, men earn around 22.22% more than women annually.


## Exercise 5

We just compared all full-time working men to all full-time working women. For this to be an accurate *causal* estimate of the effect of being a woman in the work place, what must be true of these two groups? What is one reason that this may *not* be true?

>- For this to be an accurate *causal* estimate of the effect of being a woman in the work place, there must be no difference between the full-time working men group and full-time working women group except their gender. In another word, there is no average difference in annual wage between full-time working people who we actually observe being women and full-time working people who we actually observe being men in a world where no one is women (no baseline difference). The two groups we are comparing must be counterfactuals for one and another. 
>- One reason that this may not be true may be due to the fact that there may be difference between the full-time working men group and full-time working women besides their gender due to working industry difference. Women may work in non-STEM industires that pay lower annual wage than men on average. 

## Exercise 6

One answer to the second part of Exercise 5 is that working women are likely to be younger, since a larger portion of younger women are entering the workforce as compared to older generations.

To *control* for this difference, let's now regress annual earnings on gender, age, and age-squared (the relationship between age and income is generally non-linear). What is the implied average annual wage difference between women and men? Is it different from your raw estimate? 

In [99]:
print({f"Checking data types"})
print(employed_full_at_work[["sex", "age", "annual_pay"]].dtypes)

{'Checking data types'}
sex              int8
age              int8
annual_pay    float64
dtype: object


In [100]:
employed_full_at_work["age"] = employed_full_at_work["age"].astype("int64")
# create age squared variables
employed_full_at_work["age_squared"] = employed_full_at_work["age"] ** 2

In [101]:
print("Check age squared data:")
employed_full_at_work["age_squared"]

Check age squared data:


2         2704
3          361
4         3136
6         2304
17        3481
          ... 
302318    3364
302319    1600
302326    5329
302328    1024
302329    3481
Name: age_squared, Length: 133814, dtype: int64

In [102]:
# Checking if there are any negative numbers in 'age_squared'
has_negatives = (employed_full_at_work["age_squared"] < 0).any()

if has_negatives:
    print("There are negative numbers in 'age_squared'.")
else:
    print("There are no negative numbers in 'age_squared'.")

There are no negative numbers in 'age_squared'.


In [103]:
# Checking if there are any infinite numbers in 'age_squared'
has_infinite = np.isinf(employed_full_at_work["age_squared"]).any()

if has_infinite:
    print("There are infinite numbers in 'age_squared'.")
else:
    print("There are no infinite numbers in 'age_squared'.")

There are no infinite numbers in 'age_squared'.


In [104]:
print(f"Checking missing vlaue in the dataset")
print(employed_full_at_work[["sex", "age", "age_squared", "annual_pay"]].isnull().sum())

Checking missing vlaue in the dataset
sex                0
age                0
age_squared        0
annual_pay     11211
dtype: int64


In [105]:
print({f"Checking data types"})
print(employed_full_at_work[["sex", "age", "age_squared", "annual_pay"]].dtypes)

{'Checking data types'}
sex               int8
age              int64
age_squared      int64
annual_pay     float64
dtype: object


In [106]:
# Drop rows with missing values
employed_full_at_work.dropna(subset=["annual_pay"], inplace=True)

In [107]:
# Prepare the independent variables (add a constant term for the intercept)
X = sm.add_constant(employed_full_at_work[["sex", "age", "age_squared"]])

# Prepare the dependent variable - annual_pay
y = employed_full_at_work["annual_pay"]

# Fit the linear regression model
lr_model = sm.OLS(y, X).fit()

# Display the model summary
linear_regression_model_summary = lr_model.summary()
linear_regression_model_summary

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:             annual_pay   R-squared:                       0.083
Model:                            OLS   Adj. R-squared:                  0.083
Method:                 Least Squares   F-statistic:                     3710.
Date:                Tue, 09 Apr 2024   Prob (F-statistic):               0.00
Time:                        09:49:20   Log-Likelihood:            -1.4426e+06
No. Observations:              122603   AIC:                         2.885e+06
Df Residuals:                  122599   BIC:                         2.885e+06
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
===============================================================================
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
const        3632.6906    910.464      3.990      0.000    1848.196    5417.186
sex         -1.074e+04    178.919    -60.000      0.000   -1.11e+04   -1.04e+04
age          2730.5944     41.676     65.519      0.000    2648.910    2812.279
age_squared   -25.8227      0.467    -55.283      0.000     -26.738     -24.907
==============================================================================
Omnibus:                    18004.695   Durbin-Watson:                   1.739
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            27027.927
Skew:                           1.096   Prob(JB):                         0.00
Kurtosis:                       3.699   Cond. No.                     2.39e+04
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 2.39e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

> The coefficient for sex represents the difference in the dependent variable (annual_pay) between females and the base category (males).

In [108]:
# get the coefficient for sex
coeff_sex = lr_model.params["sex"]

# Print out the coefficient for 'sex'
print(
    f"The annual wage differences between male and female is around ${abs(round(coeff_sex,2))}."
)

The annual wage differences between male and female is around $10735.1.


>- The implied average annual wage difference between women and men with the same age is around $10735.1. 
>- Women earn around $10621.15 less than men with the same age on average. 

>- It is different from my raw estimate, which is around $10514.44. 
>- My raw estimate is that women earn around $10514.44 less than men on average.

## Exercise 7

In running this regression and interpreting the coefficient on `female`, what is the implicit comparison you are making? In other words, when we run this regression and interpreting the coefficient on `female`, we're basically pretending we are comparing two groups and assuming they are counter-factuals for one another. What are these two groups?

>- In running this regression and interpreting the coefficient on `female`, the implicit comparison I'm making is that the two groups we are comparing, full time working men and full time working women with the same age, differ only by gender. 
>- We're basically pretending we are comparing full time working men and full time working women with the same age that differ only by gender and assuming they are counter-factuals for one another. 

## Exercise 8

Now let's add to our regression an indicator variable for whether the respondent has at least graduated high school, and an indicator for whether the respondent at least has a BA. 

In answering this question, use the following table of codes for the variable `grade92`. 

Education is coded as follows:
    
![CPS Educ Codes](../images/cps_educ_codes.png)

In [109]:
# Create the high school graduation indicator
employed_full_at_work["hs"] = (employed_full_at_work["grade92"] >= 39).astype(int)

# Create the bachelor's degree indicator
employed_full_at_work["ba"] = (employed_full_at_work["grade92"] >= 43).astype(int)

# Add these indicators to the independent variables
X = sm.add_constant(employed_full_at_work[["sex", "age", "age_squared", "hs", "ba"]])

# Fit the linear regression model
lr_model_new = sm.OLS(y, X).fit()

# Display the model summary
linear_regression_new_model_summary = lr_model_new.summary()
print(linear_regression_new_model_summary)

# Get the coefficients for the new indicators
coeff_hs = lr_model_new.params["hs"]
coeff_ba = lr_model_new.params["ba"]

# Print out the coefficients for the new education indicators
print(f"Coefficient for high school graduation: {coeff_hs}")
print(f"Coefficient for having at least a bachelor's degree: {coeff_ba}")

                            OLS Regression Results                            
Dep. Variable:             annual_pay   R-squared:                       0.273
Model:                            OLS   Adj. R-squared:                  0.273
Method:                 Least Squares   F-statistic:                     9195.
Date:                Tue, 09 Apr 2024   Prob (F-statistic):               0.00
Time:                        09:49:20   Log-Likelihood:            -1.4284e+06
No. Observations:              122603   AIC:                         2.857e+06
Df Residuals:                  122597   BIC:                         2.857e+06
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
const       -6163.0033    863.340     -7.139      

## Exercise 9 

In running this regression and interpreting the coefficient on `female`, what is the implicit comparison you are making? In other words, when we run this regression and interpreting the coefficient on `female`, we are once more basically pretending we are comparing two groups and assuming they are counter-factuals for one another. What are these two groups?

>- In running this regression and interpreting the coefficient on `female`, the implicit comparison I'm making is that the two groups we are comparing, full time working men and full time working women with the same age and same level of education, differ only by gender. 
>- We're basically pretending we are comparing full time working men and full time working women with the same age and same level of education that differ only by gender and assuming they are counter-factuals for one another. 

## Exercise 10

Given how the coefficient on `female` has changed between Exercise 6 and Exercise 8, what can you infer about the educational attainment of the women in your survey data (as compared to the educational attainment of men)?

In [110]:
# get the coefficient for sex
coeff_sex_new = lr_model_new.params["sex"]

# Print out the coefficient for 'sex'
print(
    f"With control for age, The annual wage differences",
    f"between male and female is around ${abs(round(coeff_sex, 2))} in Ex 6.",
)

# Print out the coefficient for 'sex'
print(
    f"With control for education and age, the annual wage differences",
    f"between male and female is around ${abs(round(coeff_sex_new, 2))} in Ex 8.",
)


# Compare the wage gap change
if abs(coeff_sex) > abs(coeff_sex_new):
    print("The wage gap decreased after accounting for educational attainment.")
else:
    print("The wage gap increased after accounting for educational attainment.")

With control for age, The annual wage differences between male and female is around $10735.1 in Ex 6.
With control for education and age, the annual wage differences between male and female is around $13038.59 in Ex 8.
The wage gap increased after accounting for educational attainment.


>- The wage gap between men and women with the same age increased after accounting for educational attainment. For individuals with the same level of education and with the same age, men are earning a lot more than women, hence widening the wage gap when education is considered alongside age.
>- The fact that the wage gap widened after accounting for education imply that women in the survey have possibly higher levels of educational attainment than men with the same age on average, which narraws the wage gap between men and women with same age when not controlling education. 
>- Wage disparity between men and women becomes more pronounced when education levels is taken into account, suggesting potential gender discrimination or other systemic factors at play in the valuation of men's and women's labor when they have similar educational backgrounds. 

## Exercise 11

What does that tell you about the *potential outcomes* of men and women before you added education as a control?

> Before adding education as control, the wage gap between men and women with same age does not fully capture the entire disparity because it overlooks the influence of education on earnings. We fail to measure the accurate causal estimate of the effect of being a women in the workplace, because there is difference between the full-time working men group and full-time working women group with the same age besides their gender due to education difference. The unobserved effect of education on annual wage is not taken into account. 

## Exercise 12

Finally, let's include *fixed effects* for the type of job held by each respondent. 

Fixed effects are a method used when we have a nested data structure in which respondents belong to groups, and those groups may all be subject to different pressures. In this context, for example, we can add fixed effects for the industry of each respondent—since wages often vary across industries, controlling for industry is likely to improve our estimates. Use `ind02` to control for industry.

(Note that fixed effects are very similar in principle to hierarchical models. There are some differences [you will read about](../fixed_effects_v_hierarchical.ipynb) for our next class, but they are designed to serve the same role, just with slightly different mechanics). 

When we add fixed effects for groups like this, our interpretation of the other coefficients changes. Whereas in previous exercises we were trying to explain variation in men and women's wages *across all respondents*, we are now effectively comparing men and women's wages *within each employment sector*. Our coefficient on `female`, in other words, now tells us how much less (on average) we would expect a woman to be paid than a man *within the same industry*, not across all respondents. 

(Note that running this regression will result in lots of coefficients popping up you don't care about. We'll introduce some more efficient methods for adding fixed effects that aren't so messy in a later class -- for now, you can ignore those coefficients!)

In [111]:
# Create the formula for the regression model including fixed effects for industry
formula = "annual_pay ~ sex + age + age_squared + hs + ba + C(ind02)"

# Fit the regression model with fixed effects for industry
fixed_effects_model = smf.ols(formula, data=employed_full_at_work).fit()

# Display the model summary
print(fixed_effects_model.summary())

                            OLS Regression Results                            
Dep. Variable:             annual_pay   R-squared:                       0.320
Model:                            OLS   Adj. R-squared:                  0.319
Method:                 Least Squares   F-statistic:                     218.9
Date:                Tue, 09 Apr 2024   Prob (F-statistic):               0.00
Time:                        09:49:25   Log-Likelihood:            -1.4243e+06
No. Observations:              122603   AIC:                         2.849e+06
Df Residuals:                  122339   BIC:                         2.852e+06
Df Model:                         263                                         
Covariance Type:            nonrobust                                         
                                                                                                                               coef    std err          t      P>|t|      [0.025      0.975]
---------------------

## Exercise 13

Now that we've added industry fixed effects, what groups are we implicitly treated as counter-factuals for one another now? 

>- In running this regression and interpreting the coefficient on `female`, the implicit comparison I'm making is that the two groups we are comparing, full time working men and full time working women with the same age, same level of education and within the same industry, differ only by gender. 
>- We're basically pretending we are comparing full time working men and full time working women with the same age, same level of education and within the same industry that differ only by gender and assuming they are counter-factuals for one another. 

## Exercise 14

What happened to your estimate of the gender wage gap when you added industry fixed effects? What does that tell you about the industries chosen by women as opposed to men?

When you're done, please come read [this discussion](discussion_regressions_incomeineq.ipynb).

In [112]:
# Get the coefficient for 'sex'
coeff_sex_fix = fixed_effects_model.params["sex"]
# Print out the coefficient for 'sex' with controls
print(
    f"With control for education, age, and industry,",
    f"the annual wage difference between male and female",
    f"within the same industry is around ${abs(round(coeff_sex_fix, 2))}.",
)

if abs(coeff_sex_new) > abs(coeff_sex_fix):
    print("The wage gap decreased after adding industry fixed effects.")
else:
    print("The wage gap increased after adding industry fixed effects.")

With control for education, age, and industry, the annual wage difference between male and female within the same industry is around $10980.28.
The wage gap decreased after adding industry fixed effects.


> The wage gap between men and women with the same age and education decreased after adding industry fixed effects. Therefore, industry choices enlarge the wage gap between men and women with same age and education. I can infer that industries chosen by women tend to have lower wage than industries chosen by men, which increases the wage difference between the two groups on average. 